In [1]:
import notebook_loader
from model import multiple_layer_model as mlp

importing notebook from /data1/dhyoon/tensor_work/book_main/model/multiple_layer_model.ipynb


In [2]:
mlp.random_fix = True

In [3]:
import numpy as np
import time

In [4]:
class SudokuDataset(mlp.Dataset):
    pass

def sudoku_init(self, max_cnt=0, train_ratio=0.8, valid_ratio=0.05):
    mlp.Dataset.__init__(self, "sudoku")
    
    if mlp.random_fix: np.random.seed(3456)
    
    xs, ys = load_sudoku_file("./data/sudoku.csv", max_cnt)

    data_count = len(xs)
    tr_count = int(data_count * train_ratio)
    va_count = int(data_count * valid_ratio)
    te_count = data_count - tr_count - va_count
    print("sudoku data was prepared {}+{}+{}={} items". \
         format(tr_count, va_count, te_count, data_count))
    te_start = tr_count + va_count

    indices = np.arange(data_count)
    np.random.shuffle(indices)

    self.train_xs = xs[indices[0:tr_count]]
    self.train_ys = ys[indices[0:tr_count]]
    self.validate_xs = xs[indices[tr_count:te_start]]
    self.validate_ys = ys[indices[tr_count:te_start]]
    self.test_xs = xs[indices[te_start:]]
    self.test_ys = ys[indices[te_start:]]

    self.input_dim = 81
    self.output_dim = 81 * 9
    self.train_count = tr_count
    
SudokuDataset.__init__ = sudoku_init

In [5]:
def load_sudoku_file(filepath, max_cnt):
    quizzes, solutions = [], []

    for line in open(filepath):
        if line[0] == 'q': continue
        quiz, solution = unpack_map(line)
        quizzes.append(quiz)
        solutions.append(solution)
        if max_cnt > 0 and len(quizzes) >= max_cnt: break

    xs = np.asarray(quizzes)
    ones = np.ones([81]).astype(int)
    solution_idxs = solutions - ones
    ys = np.eye(9)[solution_idxs].reshape(-1, 81*9)

    return xs, ys
    
def unpack_map(line):
    quiz = np.zeros([81], dtype=np.int8)
    solution = np.zeros([81], dtype=np.int8)
    for n in range(81):
        quiz[n] = int(line[n])
        solution[n] = int(line[n+82])
    return quiz, solution

In [6]:
class SudokuNotfModel(mlp.NoTFMLPModel):
    pass

def sudoku_notf_init(self, name, dset, hdims, learning_rate=0.01):
    mlp.NoTFMLPModel.__init__(self, name, dset, hdims, learning_rate)

SudokuNotfModel.__init__ = sudoku_notf_init

In [7]:
def sudoku_notf_train(self, epoch_count=10, batch_size=10):
    if batch_size == 0:
        batch_size = self.dataset.train_count
        
    batch_count = int(self.dataset.train_count / batch_size)
    report_period = epoch_count / 10
    
    if mlp.random_fix: np.random.seed(1945)
    
    time1 = time2 = int(time.time())
    
    print("Model {} train report:".format(self.name))
    
    dset = self.dataset

    for epoch in range(epoch_count):
        costs, total_accs, blank_accs, max_accs =  [], [], [], []
        
        for n in range(batch_count):
            tr_X, tr_Y = dset.get_train_data(batch_size, n)
            cost, acc_pack = self.train_step(tr_X, tr_Y)
            total_acc, blank_acc, max_acc = acc_pack
            costs.append(cost)
            total_accs.append(total_acc)
            blank_accs.append(blank_acc)
            max_accs.append(max_acc)
        
        if (epoch+1) % report_period == 0:
            va_X, va_Y = dset.get_test_data(True, 30)
            total_acc, blank_acc, max_acc = self.get_accuracy(va_X, va_Y, "acc")
            time3 = int(time.time())
            print("    Epoch {}: cost={:5.3f} ({}/{} secs) \n \
    accuracy={:5.3f}/{:5.3f}/{:5.3f}({:5.3f}/{:5.3f}/{:5.3f})". \
            format(epoch+1,np.mean(costs),time3-time2,time3-time1, \
                   np.mean(total_accs), np.mean(blank_accs), \
                   np.mean(max_accs),total_acc,blank_acc,max_acc))
            time2 = time3

    path = "params/{}.ntpt".format(self.name)
    self.save_parameters(path)

SudokuNotfModel.train = sudoku_notf_train

In [15]:
def sudoku_notf_demonstrate(self, cnt=1, detail=False):
    path = "params/{}.ntpt.npz".format(self.name)
    self.restore_parameters(path)
    
    print("Model {} Demonstration".format(self.name))

    total_blank_count = 0
    total_corr_count = 0
    
    for n in range(cnt):
        demo_X, demo_Y = self.dataset.get_test_data(False, 1)

        blank_count = 81 - np.count_nonzero(demo_X[0])
        wrong_count = 0

        if detail:
            y_n = np.argmax(np.reshape(demo_Y[0], [-1,9]), axis=1)
            print("x", np.reshape(demo_X[0], [-1, 9, 9]))
            print("y", np.reshape(y_n+1, [-1, 9, 9]))

        for k in range(blank_count):
            est, ans, pos = self.get_estimate_answer(demo_X, demo_Y)

            row, col = (pos[0]//9)+1, (pos[0]%9)+1
            estimate, answer = int(est[0])+1, int(ans[0])+1
            msg = "correct"

            if answer != estimate:
                msg = "wrong"
                wrong_count += 1

            if detail:
                print("{}: Guess [{},{}] as {} (answer: {}) : {}". \
                     format(k+1, row, col, estimate, answer, msg))

            demo_X[0][pos] = answer + 1
   
        corr_count = blank_count - wrong_count
        corr_ratio = corr_count / blank_count
        
        if detail:
            print("{} correct guesses among {} guesses => {:5.5f}".\
                format(corr_count, blank_count, corr_ratio))

        total_blank_count += blank_count
        total_corr_count += corr_count

    corr_ratio = total_corr_count / total_blank_count
    
    print("Total: {} correct guess among {} guesses: {:5.5f}". \
           format(total_corr_count, total_blank_count, corr_ratio))
    
SudokuNotfModel.demonstrate = sudoku_notf_demonstrate

In [9]:
def softmax(logits):
    logits_9 = np.reshape(logits, [-1, 9])
    result_9 = mlp.softmax(logits_9)
    result = np.reshape(result_9, [-1, 81, 9])
    return result

def softmax_derv(labels, logits):
    labels_9 = np.reshape(labels, [-1, 9])
    logits_9 = np.reshape(logits, [-1, 9])
    derv_9 = mlp.softmax_derv(labels_9, logits_9)
    derv = np.reshape(derv_9, [-1, 81, 9])
    return derv

def softmax_cross_entropy(labels, logits):
    labels_9 = np.reshape(labels, [-1, 9])
    logits_9 = np.reshape(logits, [-1, 9])
    entropy_9 = mlp.softmax_cross_entropy(labels_9, logits_9)
    entropy = np.reshape(entropy_9, [-1, 81])
    return entropy

def softmax_cross_entropy_derv(labels, probs):
    return mlp.softmax_cross_entropy_derv(labels, probs)

In [22]:
def softmax(x):
    max_elem = np.max(x, axis=2)
    diff = (x.transpose(2,0,1) - max_elem).transpose(1,2,0)
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=2)
    probs = (exp.transpose(2,0,1) / sum_exp).transpose(1,2,0)
    return probs

def softmax_derv(x, y):
    mb_size, bd_size, nom_size = x.shape
    derv = np.ndarray([mb_size, bd_size, nom_size, nom_size])
    for n in range(mb_size):
        for k in range(bd_size):
            for i in range(nom_size):
                for j in range(nom_size):
                    derv[n, k, i, j] = -y[n,k,i] * y[n,k,j]
                derv[n, k, i, i] += y[n,k,i]
    return derv

def softmax_cross_entropy(p, q):
    return -np.sum(p * np.log(q), axis=2)

def softmax_cross_entropy_derv(p, q):
    return -p / q

In [10]:
def sudoku_notf_train_step(self, x, y):
    output = self.proc_forward(x)

    loss_grad = 1.0
    
    labels = np.reshape(y, [-1, 81, 9])
    logits = np.reshape(output, [-1, 81, 9])
    probs = softmax(logits)
    entropy = softmax_cross_entropy(labels, probs)
    loss = np.mean(entropy)

    ent_grad = loss_grad / np.prod(entropy.shape)
    probs_derv = softmax_cross_entropy_derv(labels, probs)
    probs_grad = probs_derv * ent_grad
    output_derv = softmax_derv(logits, probs)
    #print("np.shape(probs_grad)", np.shape(probs_grad))
    #print("np.shape(output_derv)", np.shape(output_derv))
    probs_grad_flat = np.reshape(probs_grad, [-1, 9])
    output_derv_flat = np.reshape(output_derv, [-1, 9, 9])
    output_grad_flat = [np.matmul(output_derv_flat[n], probs_grad_flat[n]) \
                       for n in range(output.shape[0]*81)]
    output_grad = np.reshape(output_grad_flat, [-1, 729])
    self.proc_backward(x, output_grad)
    
    return loss, self.eval_accuracy(x, y, output, "acc")

SudokuNotfModel.train_step = sudoku_notf_train_step

In [11]:
def sudoku_notf_get_accuracy(self, x, y, mode="old"):
    output = self.proc_forward(x)
    return self.eval_accuracy(x, y, output, mode)

def sudoku_notf_get_estimate_answer(self, x, y):
    output = self.proc_forward(x)
    return self.eval_accuracy(x, y, output, "est")

def sudoku_notf_eval_accuracy(self, x, y, output, mode="old"):
    labels = np.reshape(y, [-1, 81, 9])
    logits = np.reshape(output, [-1, 81, 9])
    estimate = np.argmax(logits, 2)
    answer = np.argmax(labels, 2)
    correct = np.equal(estimate, answer)
    total_acc = np.mean(correct)
    
    if mode == "old": return total_acc
    
    bl_mask = 1 - np.sign(x)
    bl_correct = bl_mask * correct
    blank_acc = np.sum(bl_correct) / np.sum(bl_mask)

    probs = softmax(logits)
    max_probs = np.max(probs, 2)
    bl_probs = bl_mask * max_probs
    max_pos = np.argmax(bl_probs, 1)
    max_mask = np.eye(81)[max_pos]
    max_est = np.sum(estimate*max_mask, 1)
    max_ans = np.sum(answer*max_mask, 1)
    
    if mode == "est": return max_est, max_ans, max_pos

    max_correct = np.equal(max_est, max_ans)
    max_acc = np.mean(max_correct)
    
    if mode == "acc": return total_acc, blank_acc, max_acc

SudokuNotfModel.get_accuracy = sudoku_notf_get_accuracy
SudokuNotfModel.get_estimate_answer = sudoku_notf_get_estimate_answer
SudokuNotfModel.eval_accuracy = sudoku_notf_eval_accuracy

In [12]:
sd1 = SudokuDataset(max_cnt=1000)

sm1 = SudokuNotfModel("sudoku-notf-1", sd1, [256])
sm1.train(epoch_count=100)

# 속도가 수십배 느려짐, GPU 이용 때문인지 원인 분석 필요

sudoku data was prepared 800+50+150=1000 items
Model sudoku-notf-1 train report:
    Epoch 10: cost=2.199 (53/53 secs) 
     accuracy=0.123/0.123/0.139(0.110/0.108/0.167)
    Epoch 20: cost=2.187 (55/108 secs) 
     accuracy=0.138/0.137/0.170(0.118/0.116/0.100)
    Epoch 30: cost=2.177 (54/162 secs) 
     accuracy=0.152/0.150/0.194(0.123/0.123/0.167)
    Epoch 40: cost=2.166 (55/217 secs) 
     accuracy=0.165/0.160/0.226(0.118/0.114/0.167)
    Epoch 50: cost=2.156 (54/271 secs) 
     accuracy=0.177/0.172/0.259(0.135/0.133/0.167)
    Epoch 60: cost=2.144 (55/326 secs) 
     accuracy=0.187/0.181/0.295(0.134/0.125/0.133)
    Epoch 70: cost=2.132 (54/380 secs) 
     accuracy=0.197/0.190/0.318(0.127/0.117/0.100)
    Epoch 80: cost=2.119 (55/435 secs) 
     accuracy=0.207/0.198/0.321(0.128/0.113/0.067)
    Epoch 90: cost=2.105 (54/489 secs) 
     accuracy=0.218/0.209/0.341(0.126/0.109/0.000)
    Epoch 100: cost=2.090 (55/544 secs) 
     accuracy=0.229/0.218/0.375(0.129/0.113/0.067)


In [23]:
sd1 = SudokuDataset(max_cnt=1000)

sm1 = SudokuNotfModel("sudoku-notf-1", sd1, [256])
sm1.train(epoch_count=100)

# 속도가 수십배 느려짐, GPU 이용 때문인지 원인 분석 필요

sudoku data was prepared 800+50+150=1000 items
Model sudoku-notf-1 train report:
    Epoch 10: cost=2.199 (60/60 secs) 
     accuracy=0.123/0.123/0.139(0.110/0.108/0.167)
    Epoch 20: cost=2.187 (60/120 secs) 
     accuracy=0.138/0.137/0.170(0.118/0.116/0.100)
    Epoch 30: cost=2.177 (60/180 secs) 
     accuracy=0.152/0.150/0.194(0.123/0.123/0.167)
    Epoch 40: cost=2.166 (59/239 secs) 
     accuracy=0.165/0.160/0.226(0.118/0.114/0.167)
    Epoch 50: cost=2.156 (60/299 secs) 
     accuracy=0.177/0.172/0.259(0.135/0.133/0.167)
    Epoch 60: cost=2.144 (60/359 secs) 
     accuracy=0.187/0.181/0.295(0.134/0.125/0.133)
    Epoch 70: cost=2.132 (60/419 secs) 
     accuracy=0.197/0.190/0.318(0.127/0.117/0.100)
    Epoch 80: cost=2.119 (60/479 secs) 
     accuracy=0.207/0.198/0.321(0.128/0.113/0.067)
    Epoch 90: cost=2.105 (60/539 secs) 
     accuracy=0.218/0.209/0.341(0.126/0.109/0.000)
    Epoch 100: cost=2.090 (60/599 secs) 
     accuracy=0.229/0.218/0.375(0.129/0.113/0.067)


In [16]:
# npz 확장자가 자동으로 붙을 수 있음... 현재 sudoku_demonstrate에 파일경로 변경... 어떻게 처리하는게 가장 좋은지 확인 필요
sm1.demonstrate(2, True)

Model sudoku-notf-1 Demonstration
x [[[0 0 3 8 5 0 0 0 4]
  [1 0 0 0 0 0 6 0 0]
  [0 7 8 0 0 9 0 3 0]
  [2 0 4 3 0 0 0 0 5]
  [0 0 0 0 0 0 8 2 0]
  [0 6 9 2 0 0 1 0 3]
  [0 0 1 0 9 2 0 0 0]
  [5 8 0 0 0 1 0 0 7]
  [4 0 2 5 8 0 3 0 0]]]
y [[[9 2 3 8 5 6 7 1 4]
  [1 4 5 7 2 3 6 8 9]
  [6 7 8 1 4 9 5 3 2]
  [2 1 4 3 6 8 9 7 5]
  [3 5 7 9 1 4 8 2 6]
  [8 6 9 2 7 5 1 4 3]
  [7 3 1 6 9 2 4 5 8]
  [5 8 6 4 3 1 2 9 7]
  [4 9 2 5 8 7 3 6 1]]]
1: Guess [9,9] as 9 (answer: 1) : wrong
2: Guess [5,4] as 7 (answer: 9) : wrong
3: Guess [2,9] as 2 (answer: 9) : wrong
4: Guess [1,2] as 9 (answer: 2) : wrong
5: Guess [7,1] as 3 (answer: 7) : wrong
6: Guess [8,8] as 8 (answer: 9) : wrong
7: Guess [3,5] as 9 (answer: 4) : wrong
8: Guess [9,6] as 6 (answer: 7) : wrong
9: Guess [5,5] as 9 (answer: 1) : wrong
10: Guess [3,7] as 8 (answer: 5) : wrong
11: Guess [9,2] as 8 (answer: 9) : wrong
12: Guess [5,1] as 3 (answer: 3) : correct
13: Guess [1,6] as 3 (answer: 6) : wrong
14: Guess [4,8] as 9 (answer: 7) : w

In [ ]:
sd2 = SudokuDataset(max_cnt=10000)

sm2 = SudokuNotfModel("sudoku-notf-2", sd2, [256])
sm2.train(epoch_count=10)

In [ ]:
sm2.test()
sm2.demonstrate(2, False)

In [ ]:
sm3 = SudokuNotfModel("sudoku-notf-3", sd1, [256])
sm3.train(epoch_count=1000)

In [ ]:
sm4 = SudokuNotfModel("sudoku-notf-4", sd2, [256])
sm4.train(epoch_count=1000)

In [ ]:
sm5 = SudokuNotfModel("sudoku-notf-5", sd2, [512])
sm5.train(epoch_count=1000)